In [3]:
### Dataframes, linear algebra and visualization:
import pandas as pd
from scipy.special import softmax

# Transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

print("--- Setup complete ---")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


--- Setup complete ---


In [ ]:
df = pd.read_csv("xyz.csv")
df.head()

In [ ]:
#Load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

In [ ]:
#Create the negative scores list, tokenize each review and calculate its "negativity" score:

negative_review_sentiment = []

for review in df['REVIEWS']:
    print(review)
    encoded_review = tokenizer(review, return_tensors = 'pt')
    len_encoded = len(encoded_review['ids'].detach().numpy()[0])
    print(len_encoded)

    try:
        output = model(**encoded_review)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        score = scores[0]
    
    except:
        score = -1
    
    negative_review_sentiment.append(score)


In [ ]:
#When negativity > 0.5, it is a negative review

negative_reviews_df = pd.DataFrame(negative_review_sentiment)
negative_reviews_df.columns = ['negativity']

scored_df = pd.concat([df.reset_index().drop['index'], axis = 1, negative_reviews_df], axis = 1)

scored_df['negativity_label'] = 0

scored_df.loc[scored_df['negativity'] > 0.5, 'negativity_label'] = 1